In [442]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [443]:
# read from melodyData.txt
with open('generated/melodyDataFluteShort.txt', 'r') as f:
    parts = eval(f.read())
# a =[(0, '0.25'), (0, '1.0'), (0, '3.0'), (70, '.33'), ...]


# one-hot encoding

encodings = {}
encodingIndex = 0
for part in parts:
    for note in part:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1

decodings = {}
for k, v in encodings.items():
    decodings[v] = k        

# encode everything in a

data_encoded = []
for part in parts:
    data_encoded.append([encodings[note] for note in part])


In [444]:
sequence_length = 10

X = []
Y = []
# given data_encoded, generate training data by looping 
for i in range(len(data_encoded)):
    for j in range(len(data_encoded[i]) - sequence_length):
        X.append(data_encoded[i][j:j + 10])
        Y.append(data_encoded[i][j + 10])

# split this into training and testing sets 
# we will intentionally overfit, GET RID OF THIS LATER
# TODO: separate different parts into train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# TODO: make the model itself
# benchmark: dense is 80% accuracy
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(sequence_length, 1)),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='softplus'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(X_train), np.array(Y_train), epochs=200)

Epoch 1/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.1727 - loss: 3.8508
Epoch 2/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2717 - loss: 3.2110
Epoch 3/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2683 - loss: 3.0770
Epoch 4/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2772 - loss: 3.0711
Epoch 5/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3148 - loss: 2.9495
Epoch 6/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3089 - loss: 2.8760
Epoch 7/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3527 - loss: 2.6219
Epoch 8/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3355 - loss: 2.6035
Epoch 9/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3727 - loss: 2.6175
Epoch 10/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4034 - loss: 2.3655
Epoch 11/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4048 - loss: 2.2655
Epoch 12/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

In [445]:
loss, accuracy = model.evaluate(np.array(X_test), np.array(Y_test), verbose=1)
print(f'Loss: {loss}, Accuracy: {accuracy}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7815 - loss: 2.6136  
Loss: 2.709766387939453, Accuracy: 0.7701863646507263


In [446]:
# given a sequence, generate the next note (up to ten times)
seq = X_test[0]
total_seq = seq[:]
for i in range(6):
    pred = model.predict(np.array([seq]))
    pred = np.argmax(pred)
    seq = seq[1:] + [pred]

    total_seq.append(pred)

mapping = {0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"}

for note in total_seq:
    pitch, dur = decodings[note]
    print (mapping[pitch % 12], dur)

# convert numbers to musical note letters
# 60 = C

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 578ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
C 2.00
C 2.00
C 2.00
C 2.00
C 2.00
C 2.00
C 2.00
C 2.00
C 1.00
F 0.50
C 0.25
F 0.25
A# 0.75
C 0.25
A# 0.50
C 0.25


In [447]:
'''l = []
for i in range(10):
    l.append(i)
print (l)

a = [i*i for i in l]
print (a)'''



'l = []\nfor i in range(10):\n    l.append(i)\nprint (l)\n\na = [i*i for i in l]\nprint (a)'